# Monomial matrices and their transformations

## Libraries

In [1]:
import sys; sys.path.append("../modules")
import random

import numpy as np
import sympy; sympy.init_printing()

import Permutations as pm
from Grid import *

from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

## Classes and functions

In [2]:
class MonomialMatrix(dict):
    '''Monomial matrices in a style of a pair of a diagonal matrix and a permutation matrix.'''
    def __init__(self, diag, perm):
        if not ((type(diag) is list) and (type(perm) is pm.Permutation)): raise(TypeError)
        if not len(diag) == perm.size: raise(ValueError("len(diag) != perm.size"))
        
        self['diagonal'] = diag
        self['permutation'] = perm
        self.size = len(diag)
        self.matrix = sympy.diag(*self['diagonal'])*sympy.Matrix(self['permutation'].matrix())
        
    def associate_PM(self):
        d, p = self['diagonal'], self['permutation']
        size, pinv = sum([abs(v) for v in d]), p.inverse()
    
        I, A = sympy.eye(size), sympy.zeros(0,size)
        for k in range(len(d)):
            x = abs(d[k])
            sign = 1 if d[k]==x else -1
            prev = [abs(d[pinv.act(j)]) for j in range(p.act(k))]
            abov = [abs(d[i]) for i in range(k)]
            sp, sa = sum(prev), sum(abov)
        
            for i in range(x):
                start = sp if sign == 1 else sp+x-1
                A = A.row_insert(sa+i, I.row(start+sign*i))
        #---
        img = [np.where(row > 0)[0][0] for row in np.array(A)]
        return pm.Permutation(img)
    
    def show_in_grid(self):
        hline, vline = "+---" * self.size + "+", "|   " * self.size + "|"
        for row in np.array(self.matrix):
            idx_list = np.where(row != 0)[0]
            if idx_list.size > 0:
                idx = idx_list[0]
                val = row[idx]
                val_str = "{}" if val > 0 else "\033[31m{}\033[0m"
                print(hline + "\n"
                    + vline[: 4 * idx + 2]
                    + val_str.format(abs(val))
                    + vline[4 * (idx + 1) - 1 :])
            else: print(hline + "\n" + vline)
        else: print(hline)
#---    
MM = MonomialMatrix

In [ ]:
mm = MM([-4,2,3], pm.Permutation([1,0,2])) #; print(type(mm), mm['diagonal'], mm['permutation'].act(0))
aprm = mm.associate_PM()
pmtx = sympy.Matrix(aprm.matrix())

display(mm.matrix, pmtx)
mm.show_in_grid()

In [3]:
def correctness_checking(code):
#-- Starting MM --
    a,b,c,r = code[0], code[1], code[2], code[3]
    starting_mm = MM([-a,b,c,-a,-(a-r),c,b,-(a-r),-2*r], pm.Permutation([8,2,1,5,3,7,6,0,4]))
    snum  = len(pm.cycle_decomp(starting_mm.associate_PM()))
    sdata = {'mm': starting_mm, '#cmps': snum, 'memo': ''}

#-- Resulting MM --
    flag = True
    resulting_mm, rnum, memo = None, None, ''
    if a-r>=0:
        case = "a>=r"
        if r-c == 0:
            case += ", r=c"
            resulting_mm = MM([-r,-a,-(a-r),r,a-r,-2*r], pm.Permutation([4,2,0,3,5,1]))
            rnum = len(pm.cycle_decomp(resulting_mm.associate_PM())) + 2*b
            memo = ", b={}".format(b)
        elif r-c > 0:
            case += ", r>c"
            X, Y = b%(r-c), (a-r)%(2*r)
            Xi, Yi = (r-c)-X, (2*r)-Y
            if Xi == Y:
                case += ", Xi=Y"
                resulting_mm = MM([X,c,Xi,X,c,Xi], pm.Permutation([2,1,0,5,4,3]))
                rnum = 2*np.gcd(X+c, r-X) #len(pm.cycle_decomp(resulting_mm.associate_PM()))
                memo = ", (X+c, r-X)=({},{})={}".format(X+c, r-X, np.gcd(X+c, r-X))
            elif Xi > Y:
                case += ", Xi>Y"
                Z = Xi-Y
                resulting_mm = MM([-Z,-X,-c,-Z,Y,-X,-c,-Y,-Yi], pm.Permutation([5,7,8,2,0,3,4,6,1]))
                rnum = len(pm.cycle_decomp(resulting_mm.associate_PM()))
            else:
                resulting_mm = MM([-Xi,-X,-c,-Xi,-X,-c,Y,-Y,-Yi], pm.Permutation([6,7,8,3,4,5,0,1,2]))
                rnum = len(pm.cycle_decomp(resulting_mm.associate_PM()))
        else: # r-c < 0
            case += ", r<c"
            X, Y = b%(c-r), (a-r)%(2*r)
            Xi, Yi = (c-r)-X, (2*r)-Y
            resulting_mm = MM([Xi,X,-Y,-Yi,-r,X,Xi,-r,Y], pm.Permutation([2,1,7,8,3,5,4,0,6]))
            rnum = len(pm.cycle_decomp(resulting_mm.associate_PM()))
    else: case = "(x)"
    rdata = {'mm': resulting_mm, '#cmps': rnum/2, 'memo': memo}
#----    
    return [sdata, rdata], case

## Experiments

In [31]:
R, skipped, ones = 200, 0, 0
for count in tqdm(range(R)):
    N, case = 20, "(x)"

    while case == "(x)":
        code = [random.randint(0,N) for i in range(4)]
        try:
            code[3] = code[3]%(2*code[0]+code[1]+code[2])
        except:
            print(code)
        if not (code[0]==0 or code[3] == 0):
            data = correctness_checking(code)
            if data[0][0]['#cmps']/2 == 1: ones += 1
            case = data[1]
        skipped += 1
            
    nums = [data[0][i]['#cmps'] for i in [0,1]]
    if not nums[0] == nums[1]:
        print("nums are different!! nums = {}".format(nums) )
        print("code={}, case=[ {} ]".format(code,case))
        for d in data[0]:
            d['mm'].show_in_grid() #; display(mm.matrix)
            print("# of components = {}".format(int(d['#cmps']/2))+d['memo'])
    
print("ones = {},  skipped = {}".format(ones, skipped))

  0%|          | 0/200 [00:00<?, ?it/s]

ones = 121,  skipped = 388


## scratch

In [17]:
N, case, skip_count = 20, "(x)", 0

while case == "(x)":
    code = [random.randint(0,N) for i in range(4)] #[10,10,4,8] #[8,6,2,7] #
    code[3] = code[3]%(2*code[0]+code[1]+code[2])
    if not code[3] == 0:
        data = correctness_checking(code)
        case = data[1]
        skip_count += 1
        
#-- Display --
print("skip_count={}".format(skip_count))
print("code={}, case=[ {} ]".format(code,case))
for d in data[0]:
    d['mm'].show_in_grid() #; display(mm.matrix)
    print("# of components = {}".format(int(d['#cmps']/2))+d['memo'])
    
nums = [data[0][i]['#cmps'] for i in [0,1]]
if not nums[0] == nums[1]:
    print("nums are different!! nums = {}".format(nums) )

skip_count=1
code=[20, 15, 16, 3], case=[ a>=r, r<c ]
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 20 |
+---+---+---+---+---+---+---+---+---+
|   |   | 15 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   | 16 |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   | 20 |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   | 17 |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | 16 |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | 15 |   |   |
+---+---+---+---+---+---+---+---+---+
| 17 |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   | 6 |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
# of components = 1
+---+---+---+---+---+---+---+---+---+
|   |   | 11 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   | 2 |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |